<a href="https://colab.research.google.com/github/jtlagumbay/cebqa/blob/main/reader/cebqa_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CebQA Reader Component**
Pretrained model: RoBERTa

# **Libraries**

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00


In [100]:
import pandas as pd
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaForQuestionAnswering, TrainingArguments, Trainer, XLMRobertaTokenizerFast
from huggingface_hub import login


# **Constants**

In [8]:
CEBQA_DATASET = "jhoannarica/cebquad"

# **Loading Dataset**

## Authenticate Hugging Face

In [3]:
login()


## Access dataset

In [17]:
dataset = load_dataset(CEBQA_DATASET)

train.arrow:   0%|          | 0.00/43.4M [00:00<?, ?B/s]

validation.arrow:   0%|          | 0.00/6.28M [00:00<?, ?B/s]

test.arrow:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [36]:
dataset["train"][120]

{'id': '01529-002',
 'article_id': 1529,
 'article_title': 'Tourist van nahulog kay driver nakatulog',
 'article_body': 'Nangalandig sa emergency room sa Badian District Hospital sa Brgy. Poblacion, Badian, habagatang Sugbo, ang upat ka mga turista ug drayber sa van nga ilang gisakyan human naaksidente sa alas 3:40 sa kaadlawon sa Biyernes, Nobiyembre 17, 2023, sa Brgy. Poblacion. Ang drayber nakatulog kay hayan lapoy pa kini sa iyang kapin sa 100 ka kilometro nga biyahe sa amihanang Sugbo. Hinuon minor injuries lang ang naangkon sa mga biktima busa nakagawas ra dayon sa ospital human matambali ug mahiling. Basi sa nakuhang kasayuran sa Superbalita sa Cebu gikan sa kasaligang tinubdan, nailhan ang mga biktima nga turista nga puro taga San Antonio, Tondo, Manila, nga sila si Antonietto Avila Libunao, 64, minyo; iyang asawa nga si Carmen Pacione; Lorence Pacis Paclibon , 40, minyo; ug anak niini nga si Pacomios Pacis Paclibon, 5. Samtang ang drayber nga naangol giila nga si Emeniano Jorg

In [37]:
# Initialize variables to track the longest article
longest_article = None
max_length = 0

# Iterate through each article in the train dataset
for article in dataset["train"]:
    # Concatenate article_body and context
    combined_text = article["article_body"] + article["question"]

    # Calculate the length of the combined text
    combined_length = len(combined_text)

    # Update if this article is the longest found so far
    if combined_length > max_length:
        max_length = combined_length
        longest_article = article

# Print the longest article and its length
print(f"Longest combined article length: {max_length}")
print(f"Longest article: {longest_article}")


Longest combined article length: 5911
Longest article: {'id': '00127-003', 'article_id': 127, 'article_title': 'Senado tensiyonado atol sa pag-imbestigar ni Balderas', 'article_body': 'Puno sa tensyon atol sa imbestigasyon sa Senado sa gi-raid nga ilegal nga Pogo hub niadtong Lunes, Septiyembre 9, 2024, tungod kay ang mga magbabalaod nangasuko ni dismissed Bamban, Tarlac Mayor Aretha Balderas, kinsa nagdumili sa pagtubag sa ilang mga pangutana. Samtang ang mga magbabalaod nangutana kung giunsa niya ug ang pipila ka mga miyembro sa iyang pamilya mibiya sa nasod, si Balderas nagdumili sa paghingalan sa tawo nga nagpahigayon sa ilang pag-ikyas, tungod sa kahadlok sa iyang kinabuhi. Gisuwat hinuon niya ang ngalan sa tawo sa usa ka papel sa hangyo ni Senate President Pro Tempore Stanford Baldomar. Gihangyo ni Balderas ang mga magbabalaod nga dili isulti og kusog ang ngalan. "Do not tell the senators what to do with the information. Pinagbibigyan ka namin isulat sa papel," matod ni Senate Co

# **Prepare Dataset**

## Prepare tokenizer

In [81]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
# special_tokens = {"additional_special_tokens": ["<q>", "<c>"]}

# # Add the new tokens to the tokenizer's vocabulary
# tokenizer.add_special_tokens(special_tokens)

# special_tokens = tokenizer.special_tokens_map

# Print each special token and its corresponding ID
# for token_name, token_value in special_tokens.items():
#     token_id = tokenizer.convert_tokens_to_ids(token_value)
#     print(f"{token_name}: '{token_value}' -> {token_id}")

In [48]:
print(tokenizer.model_max_length)


512


## Tokenize

In [113]:
def filter_incomplete_examples(example):
    # Ensure both "question" and "context" exist and are non-empty
    return "question" in example and example["question"] and \
           "context" in example and "text" in example["context"] and \
           example["context"]["text"] and example["answer"]["text"]

print(len(dataset["train"]))
print(len(dataset.filter(filter_incomplete_examples)["train"]))
print(len(dataset["test"]))
print(len(dataset.filter(filter_incomplete_examples)["test"]))
print(len(dataset["validation"]))
print(len(dataset.filter(filter_incomplete_examples)["validation"]))

19340
19340
5526
5526
2763
2762


In [118]:
# # Tokenize function
# def tokenize_function(data):
#     input = f"<q>{data['question']}<c>{data['context']['text']}"
#     return tokenizer(input, padding="max_length", truncation=True)

# # Apply tokenization to the dataset
# tokenized_dataset = dataset.map(tokenize_function, batched=False)

def tokenize_function(examples):
    context_text = [context.get("text", "") for context in examples.get("context", [{}])]
    question_text = examples.get("question", [""])

    tokenized_examples = tokenizer(
        question_text,
        context_text,
        truncation="only_second",  # Truncate only the context
        max_length=512,            # Limit input length
        stride=128,                # Add a sliding window
        return_overflowing_tokens=True,  # Handle long contexts
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
    offset_mapping = tokenized_examples["offset_mapping"]

    # Lists to store calculated start and end positions
    start_positions = [0] * 1000
    end_positions = [0] * 1000

    for i, offsets in enumerate(offset_mapping):
        sample_index = sample_mapping[i]
        answer = examples["answer"][sample_index]

        # Handle missing or empty answers
        if len(answer["text"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Get the answer's start and end character positions
        start_char = answer["start"]
        end_char = answer["end"]

        # print(start_char, end_char)

        # Get the sequence IDs to identify the context part
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Identify the start and end of the context
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1
        print(start_char, offsets[context_start][0])
        # Check if the answer is out of the bounds of the context
        if start_char < offsets[context_start][0] or end_char > offsets[context_end][1]:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Find start and end tokens for the answer
            start_token = next(
                idx for idx, offset in enumerate(offsets)
                if offset[0] <= start_char < offset[1]
            )
            end_token = next(
                idx for idx, offset in enumerate(offsets)
                if offset[0] < end_char <= offset[1]
            )

            start_positions.append(start_token)
            end_positions.append(end_token)

    # Add start and end positions to the tokenized examples
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions

    return tokenized_examples

# Tokenize the dataset
tokenized_datasets = dataset.filter(filter_incomplete_examples).map(tokenize_function, batched=True)


Map:   0%|          | 0/19340 [00:00<?, ? examples/s]

110 0
147 0
37 0
35 0
91 0
51 0
32 0
91 0
86 0
57 0
64 0
3 0
43 0
11 0
0 0
129 0
26 0
48 0
66 0
10 0
17 0
11 0
33 0
88 0
3 0
182 0
0 0
54 0
64 0
274 0
76 0
109 1
29 0
91 0
90 0
0 0
101 0
109 0
0 0
21 0
73 0
51 0
307 0
40 0
63 0
30 0
215 0
72 0
0 0
89 0
56 0
91 0
118 0
38 0
46 0
9 0
0 0
133 0
0 0
55 0
14 0
11 0
39 0
0 0
195 0
3 0
0 0
0 0


RuntimeError: generator raised StopIteration

In [97]:
tokenized_dataset["train"][0]

{'id': '00778-015',
 'article_id': 778,
 'article_title': 'Rugby boys sa PUJ, susihon',
 'article_body': 'Nakig-alayon karon ang Abellana Police Station sa netizen nga ni-upload sa video diin nakita ang grupo sa mga lalaking menor nga ningsakay sa publikong sakyanan nga dunay rota nga 17B paingon sa Brgy. Sto. Gertrudes, dakbayan sa Sugbo samtang nagsige’g simhot sa cellophane nga dunay sulod nga solvent. Matod ni Police Major Emeniano Don Apechi Makring, hepe sa Abellana Police Station, nga gusto nilang masayran kon kanus-a o ang petsa sa pagkuha sa video sa rugby boys, sanglit sa post niini sa Facebook account, naghisgot kini’g skywalk dapit sa Robinsons Place sa Osmeña Blvd. apan taudtaod na kining gi-demolish. Iyang gipasabot nga matag adlaw ang ilang police station way hunong sa pag-rescue sa mga batang libud-suroy ug gipang turnover sa barangay diin nahisakop ang dapit. “Actually kining atong concern regarding sa mga batan-on nga gagamit og kining plastic bag nga gibutang ang ill

## Dataset Splitting

In [102]:
train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]

train_dataset

Dataset({
    features: ['id', 'article_id', 'article_title', 'article_body', 'question', 'context', 'answer', 'context_start', 'context_end', 'answer_start', 'answer_end', 'input_ids', 'attention_mask'],
    num_rows: 19340
})

# **Model Training**

## Load Pre-Trained RoBERTa

In [99]:
model = XLMRobertaForQuestionAnswering.from_pretrained("xlm-roberta-base")


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training Argument

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

## Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()


## Saving the fine-tuned model

In [ ]:
model.save_pretrained("path/to/save-directory")
tokenizer.save_pretrained("path/to/save-directory")

# **Evaluating the model**

## Evaluating

In [ ]:
results = trainer.evaluate()
print(results)

## Inference

In [ ]:
inputs = tokenizer("Your input text here", return_tensors="pt", truncation=True, padding="max_length")
outputs = model(**inputs)
logits = outputs.logits
predicted_class = logits.argmax().item()
print(f"Predicted class: {predicted_class}")